In [8]:
%load_ext autoreload
%autoreload 2

import numpy as np
from tqdm import tqdm

import sqlalchemy as sqla
from sqlalchemy import create_engine, Column, and_
from sqlalchemy.orm import Session

from ultrack.core.database import NodeDB
from ultrack.core.export.utils import solution_dataframe_from_sql

from cells_database import Base, CellDB, TrackDB
from cells_database import add_track_ids_to_tracks_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# create a new database

new_db_path = r'D:\kasia\tracking\E6_exp\code\gardener_20\cells_database_2tables.db'

engine = create_engine(f'sqlite:///{new_db_path}')

# creates a table
Base.metadata.create_all(engine) 

In [3]:
# get engine for the original database

org_db_path = r'd:\kasia\tracking\E6_exp\code\gardener_20\data.db'
engine_org = sqla.create_engine(f'sqlite:///{org_db_path}')

In [10]:
# get a solution in a form of a dataframe

df = solution_dataframe_from_sql(f'sqlite:///{org_db_path}')
df = add_track_ids_to_tracks_df(df)

df.reset_index(inplace=True)
df

,id,parent_id,t,z,y,x,track_id,parent_track_id,root
0,1000017,-1,0,0.0,213.0,4549.0,1,-1,1.0
1,1000020,-1,0,0.0,225.0,3607.0,4,-1,4.0
2,1000021,-1,0,0.0,231.0,5027.0,5,-1,5.0
3,1000028,-1,0,0.0,270.0,3647.0,8,-1,8.0
4,1000029,-1,0,0.0,289.0,3631.0,9,-1,9.0
...,...,...,...,...,...,...,...,...,...
2021053,239010184,238010283,238,0.0,8311.0,4620.0,69589,-1,69589.0
2021054,239010185,238010284,238,0.0,8312.0,4995.0,64351,-1,64351.0
2021055,239010186,238010287,238,0.0,8317.0,4130.0,73597,73595,73595.0
2021056,239010187,238010289,238,0.0,8319.0,4072.0,73596,73595,73595.0


## Create a cells table

In [6]:
# that has to be changed to operate on the original database
# because at the moment objects not assigned to a track are not saved in the database
# the consideration is what if multiple segmentations were given to ultrack and
# there are multiple possible objects for a single cell ???

def add_cell(row):

        global session
        global session_db_org
        
        cell = CellDB(id = row['id'],
                    t =row['t'],
                    track_id = row['track_id'],
                    parent_id = row['parent_id'], 
                    row = row['y'],
                    col = row['x'])
        
        # get a mask of this cell
        cell_obj = session_db_org.query(NodeDB).filter(NodeDB.id==row['id']).first()

        cell.mask = cell_obj.pickle.mask
        
        cell.bbox_0 = int(cell_obj.pickle.bbox[0])
        cell.bbox_1 = int(cell_obj.pickle.bbox[1])
        cell.bbox_2 = int(cell_obj.pickle.bbox[2])
        cell.bbox_3 = int(cell_obj.pickle.bbox[3])

        session.add(cell)

In [9]:
# create a table of cells

tqdm.pandas(desc="Progress")

session_db_org = Session(engine_org)
session = Session(engine)

df.progress_apply(add_cell, axis=1)

session.commit()

session_db_org.close()
session.close()

Progress: 100%|██████████| 2021058/2021058 [12:50<00:00, 2621.99it/s]


## Create a tracks table

In [13]:
df_tracks = df.groupby(['track_id','parent_track_id' ,'root']).agg({'t':['min','max']})
df_tracks.reset_index(inplace=True)
df_tracks.columns = ['_'.join(col).strip('_') for col in df_tracks.columns.values]
df_tracks

,track_id,parent_track_id,root,t_min,t_max
0,1,-1,1.0,0,73
1,2,1,1.0,74,160
2,3,1,1.0,74,169
3,4,-1,4.0,0,154
4,5,-1,5.0,0,10
...,...,...,...,...,...
75013,75014,75012,75012.0,239,240
75014,75015,-1,75015.0,238,240
75015,75016,-1,75016.0,238,240
75016,75017,-1,75017.0,238,240


In [21]:
def add_track(row):

        global session
        
        track = TrackDB(track_id = row['track_id'],
                        parent_track_id = row['parent_track_id'],
                        root = row['root'],
                        t_begin = row['t_min'],
                        t_end = row['t_max'])
        

        session.add(track)

In [22]:
# create a table of tracks

session = Session(engine)  

df_tracks.apply(add_track, axis=1)

session.commit()

session.close()

## Tests

### All the cells in a family

In [11]:
# all the tracks in the family (common root)

with Session(engine) as session:
    t = session.query(TrackDB).filter(TrackDB.root==5).all()

t

[Track 5 from 0 to 10, Track 6 from 11 to 71, Track 7 from 11 to 78]

### Entire tracks table to pandas data frame

In [23]:
import pandas as pd

In [24]:
df = pd.read_sql_table('tracks', engine)

In [25]:
df

,track_id,parent_track_id,root,t_begin,t_end
0,1,-1,1,0,73
1,2,1,1,74,160
2,3,1,1,74,169
3,4,-1,4,0,154
4,5,-1,5,0,10
...,...,...,...,...,...
75013,75014,75012,75012,239,240
75014,75015,-1,75015,238,240
75015,75016,-1,75016,238,240
75016,75017,-1,75017,238,240


### All the cells in the longest track

In [15]:
# add length of the tracks
df_tracks['track_length'] = df_tracks['t_max'] - df_tracks['t_min'] + 1
df_tracks

,track_id,parent_track_id,root,t_min,t_max,track_length
0,1,-1,1.0,0,73,74
1,2,1,1.0,74,160,87
2,3,1,1.0,74,169,96
3,4,-1,4.0,0,154,155
4,5,-1,5.0,0,10,11
...,...,...,...,...,...,...
75013,75014,75012,75012.0,239,240,2
75014,75015,-1,75015.0,238,240,3
75015,75016,-1,75016.0,238,240,3
75016,75017,-1,75017.0,238,240,3


In [17]:
# sort by length
df_tracks.sort_values(by='track_length', ascending=False)

,track_id,parent_track_id,root,t_min,t_max,track_length
18327,18328,-1,18328.0,0,240,241
25105,25106,-1,25106.0,0,240,241
4525,4526,-1,4526.0,0,240,241
1877,1878,-1,1878.0,0,240,241
22846,22847,22845,22845.0,3,240,238
...,...,...,...,...,...,...
36520,36521,-1,36521.0,82,82,1
24949,24950,24949,24919.0,149,149,1
8439,8440,8438,8438.0,25,25,1
8441,8442,8440,8438.0,26,26,1


In [18]:
# all the cells belonging to the longest track
with Session(engine) as session:
    t = session.query(CellDB).filter(CellDB.track_id==18328).all()

len(t)

241

### Get a single track by track_id

In [22]:
with Session(engine) as session:
    t = session.query(TrackDB).get(18328)

t

Track 18328 from 0 to 240

### Get all descendants